In [1]:
import tkinter as tk
import random
from PIL import Image, ImageTk
import os
import numpy as np

In [2]:
CARD_VALUES = {
    '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10,
    'J': 10, 'Q': 10, 'K': 10, 'A': 11
}

# Load card images
def load_card_images():
    images = {}
    suits = ['hearts', 'diamonds', 'clubs', 'spades']
    ranks = list(CARD_VALUES.keys())
    for suit in suits:
        for rank in ranks:
            filename = f'cards/{rank}_of_{suit}.png'
            if os.path.exists(filename):
                image = Image.open(filename)
                image = image.resize((72, 96), Image.LANCZOS)
                images[f'{rank}_of_{suit}'] = ImageTk.PhotoImage(image)
    # Load back of card image
    back_image = Image.open('cards/back.png')
    back_image = back_image.resize((72, 96), Image.LANCZOS)
    images['back'] = ImageTk.PhotoImage(back_image)
    return images

# Create deck of cards
def create_deck():
    suits = ['hearts', 'diamonds', 'clubs', 'spades']
    ranks = list(CARD_VALUES.keys())
    deck = []
    for suit in suits:
        for rank in ranks:
            deck.append(f'{rank}_of_{suit}')
    random.shuffle(deck)
    return deck

# Calculate hand value
def calculate_hand(hand):
    value = 0
    aces = 0
    for card in hand:
        rank = card.split('_of_')[0]
        value += CARD_VALUES.get(rank, 0)
        if rank == 'A':
            aces += 1
    while value > 21 and aces:
        value -= 10
        aces -= 1
    return value

In [3]:
class BlackjackGame:
    def __init__(self, master):
        self.master = master
        self.master.title("Blackjack")
        self.master.geometry("400x575")  # Set fixed window size
        self.master.resizable(False, False)  # Disable window resizing
        self.balance = 10
        self.card_images = load_card_images()
        self.create_widgets()
        self.deck = []
        self.player_hand = []
        self.dealer_hand = []
        self.bet = 0

    def create_widgets(self):
        self.playing_field_color = "#228B22"  # Green color for playing field and toolbar

        self.master.config(bg=self.playing_field_color)

        self.balance_label = tk.Label(self.master, text=f"Balance: {self.balance} euros", bg=self.playing_field_color)
        self.balance_label.pack(pady=10)

        self.bet_label = tk.Label(self.master, text="Place your bet (1-5000 euros):", bg=self.playing_field_color)
        self.bet_label.pack(pady=10)

        self.bet_entry = tk.Entry(self.master)
        self.bet_entry.pack(pady=10)

        self.bet_button = tk.Button(self.master, text="Place Bet", command=self.place_bet)
        self.bet_button.pack(pady=10)

        self.dealer_frame = tk.Frame(self.master, bg=self.playing_field_color)
        self.dealer_frame.pack(pady=10)
        self.dealer_label = tk.Label(self.dealer_frame, text="Dealer's Hand:", anchor='center', bg=self.playing_field_color)
        self.dealer_label.pack(side=tk.LEFT)
        self.dealer_cards = tk.Frame(self.dealer_frame, bg=self.playing_field_color)
        self.dealer_cards.pack(side=tk.LEFT)

        self.player_frame = tk.Frame(self.master, bg=self.playing_field_color)
        self.player_frame.pack(pady=10)
        self.player_label = tk.Label(self.player_frame, text="Your Hand:", anchor='center', bg=self.playing_field_color)
        self.player_label.pack(side=tk.LEFT)
        self.player_cards = tk.Frame(self.player_frame, bg=self.playing_field_color)
        self.player_cards.pack(side=tk.LEFT)

        self.hit_button = tk.Button(self.master, text="Hit", command=self.player_hit)
        self.stand_button = tk.Button(self.master, text="Stand", command=self.player_stand)

        self.message_label = tk.Label(self.master, text="", bg=self.playing_field_color)
        self.message_label.pack(pady=10)

        self.reset_game()   
        self.hit_button.pack_forget()
        self.stand_button.pack_forget()

    def place_bet(self):
        try:
            self.bet = int(self.bet_entry.get())
            if self.bet < 0 or self.bet > 5000 or self.bet > self.balance:
                self.message_label.config(text="Invalid bet amount.")
            elif self.bet == 0:
                self.deck = create_deck()
                self.message_label.config(text="Cards have been reshuffled.")
            else:
                self.balance -= self.bet
                self.balance_label.config(text=f"Balance: {self.balance} euros")
                self.bet_entry.delete(0, tk.END)
                self.start_game()
        except ValueError:
            self.message_label.config(text="Please enter a valid bet amount.")

    def start_game(self):
        self.message_label.config(text="")
        self.deck = create_deck()
        self.player_hand = []
        self.dealer_hand = []

        self.player_hand.append(self.deck.pop())
        self.dealer_hand.append(self.deck.pop())
        self.player_hand.append(self.deck.pop())
        self.dealer_hand.append(self.deck.pop())

        self.update_hand_labels()
        self.hit_button.pack()
        self.stand_button.pack()

    def update_hand_labels(self):
        # Update player hand
        for widget in self.player_cards.winfo_children():
            widget.destroy()
        player_value = calculate_hand(self.player_hand)
        for card in self.player_hand:
            img = self.card_images.get(card, self.card_images['back'])
            tk.Label(self.player_cards, image=img, bg=self.playing_field_color).pack(side=tk.LEFT)
        self.player_label.config(text=f"Your Hand: {player_value}")

        # Update dealer hand
        for widget in self.dealer_cards.winfo_children():
            widget.destroy()
        # Show only first dealer card
        first_card = self.dealer_hand[0]
        img = self.card_images.get(first_card, self.card_images['back'])
        tk.Label(self.dealer_cards, image=img, bg=self.playing_field_color).pack(side=tk.LEFT)
        # Show back image for hidden card
        tk.Label(self.dealer_cards, image=self.card_images['back'], bg=self.playing_field_color).pack(side=tk.LEFT)
        # Calculate the value of the revealed card only
        revealed_value = CARD_VALUES[first_card.split('_of_')[0]]
        self.dealer_label.config(text=f"Dealer's Hand: {revealed_value}")

    def player_hit(self):
        self.player_hand.append(self.deck.pop())
        self.update_hand_labels()
        if calculate_hand(self.player_hand) > 21:
            self.message_label.config(text="You bust! Dealer wins.")
            self.end_round()

    def player_stand(self):
        self.hit_button.pack_forget()
        self.stand_button.pack_forget()
        self.dealer_turn()

    def dealer_turn(self):
        dealer_value = calculate_hand(self.dealer_hand)
        while dealer_value < 17:
            self.dealer_hand.append(self.deck.pop())
            dealer_value = calculate_hand(self.dealer_hand)
        
        self.show_final_hands()
        self.determine_winner()

    def show_final_hands(self):
        for widget in self.dealer_cards.winfo_children():
            widget.destroy()
        for card in self.dealer_hand:
            img = self.card_images.get(card, self.card_images['back'])
            tk.Label(self.dealer_cards, image=img, bg=self.playing_field_color).pack(side=tk.LEFT)
        dealer_value = calculate_hand(self.dealer_hand)
        self.dealer_label.config(text=f"Dealer's Hand: {dealer_value}")

    def determine_winner(self):
        player_value = calculate_hand(self.player_hand)
        dealer_value = calculate_hand(self.dealer_hand)

        if dealer_value > 21 or (player_value > dealer_value and player_value <= 21):
            winnings = self.bet * 2
            self.message_label.config(text=f"You win! You get {winnings} euros.")
            self.balance += winnings
        elif player_value == dealer_value:
            self.message_label.config(text=f"It's a tie! You get your bet of {self.bet} euros back.")
            self.balance += self.bet
        else:
            self.message_label.config(text=f"Dealer wins. You lose {self.bet} euros.")
        self.balance_label.config(text=f"Balance: {self.balance} euros")
        self.end_round()

    def end_round(self):
        self.hit_button.pack_forget()
        self.stand_button.pack_forget()
        if self.balance > 0:
            self.bet_button.config(text="Play Again", command=self.reset_game)
        else:
            self.message_label.config(text="Game over! You have run out of money.")
            self.bet_button.config(state=tk.DISABLED)

    def reset_game(self):
        self.bet = 0
        self.bet_entry.delete(0, tk.END)
        self.bet_button.config(text="Place Bet", command=self.place_bet)
        
        # Clear player and dealer card images
        for widget in self.player_cards.winfo_children():
            widget.destroy()
        for widget in self.dealer_cards.winfo_children():
            widget.destroy()
        
        # Reset labels
        self.player_label.config(text="Your Hand:")
        self.dealer_label.config(text="Dealer's Hand:")
        self.message_label.config(text="")
        
        # Repack labels to center them
        self.player_label.pack(side=tk.TOP, anchor='center')
        self.dealer_label.pack(side=tk.TOP, anchor='center')

# Run the game
if __name__ == "__main__":
    root = tk.Tk()
    root.img = tk.PhotoImage(file='cards/red_joker.png')
    root.iconphoto(False, root.img)
    game = BlackjackGame(root)
    root.mainloop()